In [1]:
!pip install --quiet google-cloud-storage

In [2]:
from google.colab import auth, files
auth.authenticate_user()

# CONFIGURATION
BUCKET_NAME      = 'sky_long_medical_bucket'
PREFIX           = 'BUU_LSPINE_V2/BUU_LSPINE_V2/AP/' # GCS “folder” you want (EDIT THIS)
LOCAL_DIR        = '/content/AP/'           # where to store in Colab
START_INDEX      = 0                            # zero‑based index of first image (EDIT THIS)
CHUNK_SIZE       = 3600                           # how many images to grab (EDIT THIS)
DOWNLOAD_LOCAL_AS_ZIP = False                   # set True to zip and browser‑download to local device

import os
from google.cloud import storage

os.makedirs(LOCAL_DIR, exist_ok=True)
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

all_blobs = list(client.list_blobs(bucket, prefix=PREFIX))
image_exts = ('.jpg','.jpeg','.png')
image_blobs = [
    b for b in all_blobs
    if not b.name.endswith('/') and b.name.lower().endswith(image_exts)
]
image_blobs.sort(key=lambda b: b.name)

end_index = START_INDEX + CHUNK_SIZE
selected = image_blobs[START_INDEX:end_index]
print(f"Downloading images {START_INDEX}–{end_index-1} "
      f"(total {len(selected)}) of {len(image_blobs)} available")

for blob in selected:
    dest = os.path.join(LOCAL_DIR, os.path.basename(blob.name))
    blob.download_to_filename(dest)
    print(f"⬇️ {blob.name} → {dest}")

# Optional ZIP and Download
if DOWNLOAD_LOCAL_AS_ZIP:
    import shutil
    zip_name = 'images'
    zip_path = shutil.make_archive(zip_name, 'zip', LOCAL_DIR)
    print(f"\n Created archive at {zip_path}")
    files.download(zip_path)

⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0001-F-037Y0.jpg → /content/AP/0001-F-037Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0003-F-013Y0.jpg → /content/AP/0003-F-013Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0004-F-010Y0.jpg → /content/AP/0004-F-010Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0005-F-025Y0.jpg → /content/AP/0005-F-025Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0006-F-031Y0.jpg → /content/AP/0006-F-031Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0007-F-006Y0.jpg → /content/AP/0007-F-006Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0008-M-022Y0.jpg → /content/AP/0008-M-022Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0009-F-038Y0.jpg → /content/AP/0009-F-038Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0010-F-037Y0.jpg → /content/AP/0010-F-037Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0011-F-063Y0.jpg → /content/AP/0011-F-063Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0012-F-013Y0.jpg → /content/AP/0012-F-013Y0.jpg
⬇️ BUU_LSPINE_V2/BUU_LSPINE_V2/AP/0013-M-022Y0.jpg → /content/AP/0013-M-022Y0.jpg
⬇️ BUU_LSPINE_V2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import measure, morphology, filters, color, util, io, feature, data, exposure
from scipy import ndimage as ndi
from skimage.util import random_noise, compare_images, invert
import skimage.morphology
from skimage.segmentation import watershed
from skimage.feature import peak_local_max, corner_harris, corner_subpix, corner_peaks
from skimage.transform import warp, AffineTransform
from skimage.draw import ellipse
from skimage.filters import meijering, sato, frangi, hessian
from skimage.measure import shannon_entropy
!pip install mahotas
import mahotas
import cv2 as cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.4 MB/s eta 0:00:00


In [4]:
from skimage.transform import resize

# Image normalization
def normalize_image(image, target_shape=(640,640)):
    image = resize(image, target_shape, anti_aliasing=True)

    image = image.astype('float32')
    image /= image.max()
    return image

In [5]:
def to_grayscale(image):
    """Convert an image to grayscale."""
    if len(image.shape) == 3:  # Check if the image is RGB
        return color.rgb2gray(image)
    return image  # Already grayscale

# Image Processing Functions
def sobel_filter(image):
    """Apply Sobel filter to the image."""
    return filters.sobel(image)

def gaussian_filter(image, sigma=1): # best result so far
    """Apply Gaussian filter to the image."""
    return filters.gaussian(image, sigma=sigma)

def laplace_filter(image):
    """Apply Laplace filter to the image."""
    return filters.laplace(image)

In [6]:
# FOR TEXTURE
# Function which returns an ndarray of texture scores for each pixel in the image (640x640)

def texture_score(image, roi_mask=None, sigma_min=1, sigma_max=16):
    # Convert to grayscale if needed
    if image.ndim == 3:
        image = color.rgb2gray(image)
    # Optionally crop/segment ROI
    if roi_mask is not None:
        image = image * roi_mask

    image = exposure.rescale_intensity(image, out_range=(0, 1))
    # Ensure image is in float format
    image = image.astype('float32')
    # Extract texture features
    features = feature.multiscale_basic_features(
        image,
        intensity=False,
        edges=False,
        texture=True,
        sigma_min=sigma_min,
        sigma_max=sigma_max,
        channel_axis=None
    )
    return features

In [7]:
# FOR EDGE
# Function which returns an ndarray of edge scores for each pixel in the image (640x640)

def edge_score(image, roi_mask=None, sigma_min=1, sigma_max=16):
    # Convert to grayscale if needed
    if image.ndim == 3:
        image = color.rgb2gray(image)
    # Optionally crop/segment ROI
    if roi_mask is not None:
        image = image * roi_mask
    # Extract edge features
    features = feature.multiscale_basic_features(
        image,
        intensity=False,
        edges=True,
        texture=False,
        sigma_min=sigma_min,
        sigma_max=sigma_max,
        channel_axis=None
    )
    return features

In [8]:
# Function which returns an ndarry of texture and edge scores for each pixel in the image (640x640)

def texture_edge_score(image, roi_mask=None, sigma_min=1, sigma_max=16):
    # Convert to grayscale if needed
    image = color.rgb2gray(image) if image.ndim == 3 else image
    # Optionally crop/segment ROI
    if roi_mask is not None:
        image = image * roi_mask
    # Extract texture and edge features
    features = feature.multiscale_basic_features(
        image,
        intensity=False,
        edges=True,
        texture=True,
        sigma_min=sigma_min,
        sigma_max=sigma_max,
        channel_axis=None
    )
    return features

In [9]:
# FOR SHAPE
# Function which returns an ndarry of shape scores for each pixel in the image (640x640)

from skimage.feature import shape_index

def shape_score(image, roi_mask=None):
    # Convert to grayscale if needed
    image = color.rgb2gray(image) if image.ndim == 3 else image
    # Optionally crop/segment ROI
    if roi_mask is not None:
        image = image * roi_mask

    image = exposure.rescale_intensity(image, out_range=(0, 1))
    # Compute shape index
    features = shape_index(image, sigma=0.1)
    return features

In [10]:
# function to normalise and calculate scores from each index in the ndarray

def calculate_score(features):
    # features: ndarray of shape (height, width, n_features)
    # Flatten to 2D: (num_pixels, n_features)
    flat = features.reshape(-1, features.shape[-1])
    # Compute mean feature value per pixel
    pixel_means = np.nanmean(flat, axis=1)
    # Min-max normalization to [0, 10]
    min_val, max_val = pixel_means.min(), pixel_means.max()
    if max_val == min_val:
        normalized = np.zeros_like(pixel_means)
    else:
        normalized = 10 * (pixel_means - min_val) / (max_val - min_val)
    # Return overall texture score (mean of normalized values)
    return normalized.mean()

In [11]:
# Function used to display calculations for texture, edge, texture-edge, and shape scores

def display_calculations(image, title):
    print(f"Calculating scores for {title}...")
    texture = texture_score(image)
    edge = edge_score(image)
    texture_edge = texture_edge_score(image)
    shape = shape_score(image)

    texture_mean = calculate_score(texture)
    print(f"Texture score (0-10):", texture_mean)

    edge_mean = calculate_score(edge)
    print(f"Edge score (0-10):", edge_mean)

    texture_edge_mean = calculate_score(texture_edge)
    print(f"Texture edge score (0-10):", texture_edge_mean)

    shape_val = calculate_score(shape)
    print(f"Shape score (0-10):", shape_val)
    return [texture_mean, edge_mean, shape_val]

In [12]:
# FOR TEXTURE
def haralick_score(image, all_features=True):

    image = color.rgb2gray(image) if image.ndim == 3 else image # ensure grayscale
    if image.dtype != np.uint8:
        image = (image / image.max() * 255)
    filtered = mahotas.gaussian_filter(image, 4)
    filtered_uint8 = filtered.astype(np.uint8)

    # Show the filtered image
    #print("Filtered Image")
    #imshow(filtered_uint8)
    #show()

    haralick_names = [
        "Angular Second Moment",
        "Contrast",
        "Correlation",
        "Variance",
        "Homogenity",
        "Sum Average",
        "Sum Variance",
        "Sum Entropy",
        "Entropy",
        "Difference Variance",
        "Difference Entropy",
        "Info Measure Corr 1",
        "Info Measure Corr 2"
    ]

    chosen_haralick_features = [
        "Entropy",                  # f9
        "Contrast",                 # f2
        "Difference Entropy",       # f11
        "Homogenity",               # f5
        "Correlation"               # f3
    ]

    # Compute Haralick features
    h = mahotas.features.haralick(filtered_uint8)
    feature_means = h.mean(axis=0)

    # Map selected features to their indices
    selected_indices = [haralick_names.index(f) for f in chosen_haralick_features]

    # Extract only those selected values
    selected_feature_vector = [float(feature_means[i]) for i in selected_indices]

    # Optional: Debug prints
    #print("Chosen Haralick Features:", chosen_haralick_features)
    #print("Selected Feature Vector:", selected_feature_vector)
    #print("Haralick Features Matrix Shape:", h.shape)
    #imshow(h)

    return selected_feature_vector if all_features==False else feature_means


In [13]:
from skimage.measure import shannon_entropy

def canny_edge_score(image):
    # Convert to grayscale if needed
    image = color.rgb2gray(image) if image.ndim == 3 else image

    # Normalize and convert to uint8 if needed
    if image.dtype != np.uint8:
        image = (image / image.max() * 255).astype(np.uint8)

    # Apply Canny edge detection
    edges = cv.Canny(image, threshold1=50, threshold2=150)

    # Edge Density = fraction of edge pixels over total pixels
    edge_density = np.sum(edges > 0) / edges.size

    # Edge Entropy = Shannon entropy of edge image
    edge_entropy = shannon_entropy(edges)

    return [edge_density, edge_entropy]

In [14]:
# FOR SHAPE
# Invariant shape descriptors (scale, rotation, translation).
# Why useful: As bones deform with age, these moments change—especially in the curvature of the spine or shape of vertebral bodies.

def moments_score(image, method='sum_log_abs'):
    # Convert to grayscale
    image = color.rgb2gray(image) if image.ndim == 3 else image

    # Convert to uint8 (required for cv2 moments)
    if image.dtype != np.uint8:
        image = (image / image.max() * 255).astype(np.uint8)

    # Apply threshold to extract shape (binary mask)
    _, binary = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # Compute Hu Moments (7 invariant values)
    moments = cv.moments(binary)
    huMoments = cv.HuMoments(moments).flatten()

    # Log-transform them (recommended)
    huMoments_log = -np.sign(huMoments) * np.log10(np.abs(huMoments) + 1e-10)

    # Choose how to summarize:
    if method == 'sum_log_abs':
        score = np.sum(np.abs(huMoments_log))
    elif method == 'mean':
        score = np.mean(huMoments_log)
    elif method == 'max':
        score = np.max(np.abs(huMoments_log))
    else:
        raise ValueError("Unsupported method")

    return score

In [15]:
# 2 different augmentations per training dataset

import albumentations as A

augment1 = A.Compose([
    A.Rotate(limit=12, p=1.0),
    A.GaussianBlur(blur_limit=(17, 31), p=1.0)
])

augment2 = A.Compose([
    A.Affine(translate_percent={"x": 0.08, "y": 0.08}, scale=(0.95, 1.05), rotate=0, p=1.0),
    A.RandomBrightnessContrast(brightness_limit=0.22, contrast_limit=0.22, p=1.0)
])

In [ ]:
# creates and adds augmented types to the training set.

def augment(df):
    augmented_rows = []

    for idx, row in df.iterrows():
        image = row['image']

        # Original row (label as 0.0)
        orig_row = row.copy()
        orig_row['augmentation_type'] = 0.0
        augmented_rows.append(orig_row)

        # Augmentation 1
        aug_img1 = augment1(image=image)['image']
        row_aug1 = row.copy()
        row_aug1['image'] = aug_img1
        row_aug1['augmentation_type'] = 1.0
        augmented_rows.append(row_aug1)

        # Augmentation 2
        aug_img2 = augment2(image=image)['image']
        row_aug2 = row.copy()
        row_aug2['image'] = aug_img2
        row_aug2['augmentation_type'] = 2.0
        augmented_rows.append(row_aug2)

    # Create new DataFrame from all rows
    df_augmented = pd.DataFrame(augmented_rows).reset_index(drop=True)

    return df_augmented

In [17]:
import os
import re
import pandas as pd
from skimage.io import imread

def process_and_save_features(image_files, batch_idx, output_dir):
    features_rows = []
    for file_path in image_files:
        file = os.path.basename(file_path)
        match = re.search(r'(\d+)-([MF])-0*(\d+)Y', file)
        if match:
            _, gender, age = match.groups()
            age = int(age)
            try:
                image = imread(file_path)
                # ---- Feature extraction ----
                norm = normalize_image(image)
                texture = calculate_score(texture_score(norm))
                edge = calculate_score(edge_score(norm))
                shape = calculate_score(shape_score(norm))
                haralick = haralick_score(norm, all_features=True)
                canny = canny_edge_score(norm)
                moments = moments_score(norm)
                row = {
                    'filename': file,
                    'sex': gender,
                    'age': age,
                    'image_angle': 'AP',
                    'texture_score': texture,
                    # Haralick features (flattened)
                    **{col: h for col, h in zip([
                      'angular_second_moment', 'contrast', 'correlation', 'variance', 'homogenity', 'sum_avg',
                      'sum_var', 'sum_entropy', 'entropy', 'diff_var', 'diff_entropy', 'im_corr_1', 'im_corr_2'
                    ], haralick)},
                    'edge_score': edge,
                    'edge_density': canny[0],
                    'edge_entropy': canny[1],
                    'shape_score': shape,
                    'moments_score': moments
                }
                features_rows.append(row)
            except Exception as e:
                print(f"Error processing {file}: {e}")
    # Save this batch to a CSV (or parquet)
    batch_df = pd.DataFrame(features_rows)
    batch_df.to_csv(os.path.join(output_dir, f'features_batch_AP_{batch_idx}.csv'), index=False)
    print(f"Saved batch {batch_idx} with {len(batch_df)} rows.")

def batch_feature_pipeline(image_folder, output_dir, batch_size=100):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]
    for i in range(0, len(image_files), batch_size):
        batch = image_files[i:i+batch_size]
        process_and_save_features(batch, i//batch_size, output_dir)

In [18]:
# ---- Run this once per folder ----
batch_feature_pipeline(
    image_folder='/content/AP',
    output_dir='/content/AP_features',
    batch_size=100
)

/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)


Saved batch 0 with 100 rows.


/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)


Saved batch 1 with 100 rows.


/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)


...

Saved batch 32 with 100 rows.


/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)


Saved batch 33 with 100 rows.
Saved batch 34 with 100 rows.


/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)
/tmp/ipython-input-10-1196435134.py:8: RuntimeWarning: Mean of empty slice
  pixel_means = np.nanmean(flat, axis=1)


Saved batch 35 with 100 rows.


In [22]:
# ---- At the end, combine all batch CSVs ----
import glob
feature_files = glob.glob('/content/AP_features/features_batch_AP_*.csv')
df_all = pd.concat([pd.read_csv(f) for f in feature_files], ignore_index=True)
df_all.to_csv('/content/AP_final_features.csv', index=False)

In [23]:
print(df_all.shape)
df_all

(3600, 23)


,filename,sex,age,image_angle,texture_score,angular_second_moment,contrast,correlation,variance,homogenity,...,entropy,diff_var,diff_entropy,im_corr_1,im_corr_2,edge_score,edge_density,edge_entropy,shape_score,moments_score
0,2962-M-032Y0.jpg,M,32,AP,5.039354,0.002913,5.566722,0.998493,1848.549094,0.636855,...,9.633120,0.001351,2.004309,-0.669421,0.999965,0.463909,0.037522,0.230813,3.226945,58.844284
1,4121-F-070Y0.jpg,F,70,AP,6.058539,0.003926,2.620241,0.999520,2729.087872,0.611739,...,9.774406,0.001517,1.940789,-0.674788,0.999974,0.522037,0.050488,0.288468,1.849532,59.380330
2,3685-M-059Y0.jpg,M,59,AP,5.933115,0.011099,2.549287,0.999539,2763.417612,0.651393,...,9.250843,0.001684,1.889971,-0.685497,0.999964,0.448006,0.046467,0.271196,2.873526,59.457433
3,2802-F-045Y0.jpg,F,45,AP,5.892860,0.106764,3.080189,0.999711,5334.174586,0.716987,...,7.707998,0.001684,1.838604,-0.708059,0.999887,0.431680,0.044648,0.263213,2.662778,59.148685
4,0861-M-075Y0.jpg,M,75,AP,6.046583,0.005560,4.107193,0.999011,2076.913886,0.566816,...,9.856573,0.001446,2.179059,-0.630114,0.999936,0.650590,0.091646,0.441939,1.379080,59.664898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2453-F-063Y0.jpg,F,63,AP,5.796998,0.003483,3.249295,0.999326,2409.918725,0.654879,...,9.527559,0.001558,1.820417,-0.687560,0.999973,0.431826,0.036775,0.227305,3.864940,60.224637
3596,3043-M-079Y0.jpg,M,79,AP,6.254015,0.001768,3.437965,0.998618,1244.260454,0.562647,...,9.845154,0.001472,2.082194,-0.615901,0.999912,1.034961,0.076714,0.390493,2.096742,60.606735
3597,4114-M-050Y0.jpg,M,50,AP,4.546186,0.001592,4.943780,0.998902,2251.839323,0.556994,...,10.134768,0.001137,2.214398,-0.626227,0.999945,0.608561,0.070759,0.368751,2.182861,59.835281
3598,0875-M-049Y0.jpg,M,49,AP,5.983630,0.006398,6.272881,0.999202,3929.605737,0.588615,...,9.983008,0.001193,2.217128,-0.646076,0.999959,0.567017,0.062610,0.337719,7.491034,59.321727
